In [1]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 4.7 MB/s eta 0:00:00


In [2]:
pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.8 MB/s eta 0:00:00


In [3]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00


In [4]:
import openai
import pinecone
from sentence_transformers import SentenceTransformer

In [5]:
!pip install --upgrade pinecone-client
import os
from pinecone import Pinecone, ServerlessSpec


In [6]:
import pinecone
# Setting OpenAI API key
openai.api_key = "sk-pro-******QlIA"

# Setting Pinecone API key
pinecone_api_key = "0a6b8f29-28b9-4bb7-83ff-583aa15d3143"
pinecone_environment = "us-west1-gcp"  # Replace with your environment
pinecone_index_name = "qa_bot_index"

# Connecting to Pinecone
pinecone_client = pinecone.Pinecone(
    api_key=pinecone_api_key,
    environment=pinecone_environment
)

In [7]:
# Function to create or get Pinecone index
def create_or_get_index():
    if pinecone_index_name not in pinecone.list_indexes():
        pinecone.create_index(index_name=pinecone_index_name)
    return pinecone_index_name

In [8]:
# Function to insert vectors into Pinecone
def insert_vectors(vectors):
    index_name = create_or_get_index()
    pinecone.upsert_items(index_name=index_name, items=vectors)

In [9]:
# Function to search Pinecone index
def search_pinecone(query_vector, num_results=1):
    index_name = create_or_get_index()
    results = pinecone.query(index_name=index_name, query_vector=query_vector, top_k=num_results)
    return results

In [10]:
# Function to generate answer using OpenAI API
def generate_answer(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.5,
        max_tokens=150,
        n=1
    )
    return response.choices[0].text.strip()

In [11]:

# Loading a pre-trained sentence embedding model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Creating Convert to Vector function

def convert_to_vector(text):
    # Generate an embedding for the given text
    vector = embedding_model.encode(text)
    return vector

In [13]:
# Main function
def main():
    # Sample question
    question = "How many countries are there in world"

    # Using OpenAI API to generate a passage related to the question
    passage = generate_answer(question)

    # Converting the passage into a vector representation
    passage_vector = convert_to_vector(passage)  # You need to implement this function

    # Inserting the passage vector into Pinecone
    insert_vectors([(passage_vector, question)])

    # Giving a new question, finding the most relevant passage vector
    new_question = "Name the largest country in terms of area."
    new_question_vector = convert_to_vector(generate_answer(new_question))  # Implementing this function
    results = search_pinecone(new_question_vector)

    # Displaying the answer from the most relevant passage
    if results:
        answer = results[0]["data"]
        print(f"Answer to '{new_question}': {answer}")
    else:
        print("No relevant answer found.")


In [14]:
# Runing the main function
main()

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.